In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:

iris = datasets.load_iris()

In [173]:
X = iris['data'][:,(2,3)]
# X = np.c_[np.ones((X.shape[0],1)),X]
y = iris['target']
y1 = (y ==2).astype(np.int)

In [174]:
# add bias to X
X = np.c_[np.ones([X.shape[0],1]), X]

In [175]:
#softmax regression implementation

#split into test, train and validation sets
#choose sizes
test_ratio = 0.2
validation_ratio = 0.2
data_len = len(X)

train_count = int(data_len * (1 - test_ratio - validation_ratio))
test_count = int(data_len * 0.2)
validation_count = int(data_len * 0.2)

assert sum([train_count, test_count,validation_count]) == data_len

scrambled_indexs = np.random.permutation(data_len)

train_ind = scrambled_indexs[:train_count]
valid_ind = scrambled_indexs[train_count:train_count+validation_count]
test_ind = scrambled_indexs[train_count+validation_count:]


#one hot encode y
def one_hot(y):
    levels = np.unique(y)
    num_levels = len(levels)
    y_onehot = np.zeros([y.shape[0],num_levels])
    for i, val in enumerate(levels):
        y_onehot[(y == val),i] = 1
    return y_onehot.astype(np.int)

#soft max function
def soft_max(logit):
    #return normalized logic vector
    logit_exp = np.exp(logit)
    normalization_const = np.sum(logit_exp, axis = 1, keepdims = True)
    return logit_exp / normalization_const

y_onehot = one_hot(y)
X_train = X[train_ind,:]
y_train = y_onehot[train_ind,:]
# batch gradient descent

eta = 0.01
m = X_train.shape[0]
theta = np.random.randn(X.shape[1],y_onehot.shape[1])
for iteration in range(5000):
    logits = X_train.dot(theta)
    y_proba = soft_max(logits)
    error = y_proba - y_train
    gradients = 1/m * X_train.T.dot(error)
    theta = theta - eta * gradients
    pass
theta  

# predicitons
y_valid_logits = X[valid_ind,:].dot(theta)
y_valid_proba = soft_max(y_valid_logits)
y_valid_pred = np.argmax(y_valid_proba, axis =1)

np.mean(y_valid_pred == np.argmax(y_onehot[valid_ind,:],axis =1))




0.90000000000000002

In [96]:
class MyLogReg(object):
    '''A one versus all classifier, currently
    without the final step of translating the probability scores to 
    predictions'''
    
    def __init__(self, eta = 0.1, n_iters = 1000):
        self.eta = eta
        self.n_iters = n_iters
    
    def sigma(t):
        return 1 / (1 + np.exp(-t))

    vsigma = np.vectorize(sigma)

    def delJ(self, m, X, theta, y):
        return 1/m * X.T.dot(self.vsigma(X.dot(theta)) - y.reshape(-1,1))

    def one_hot(self, y):
        class_labs = np.unique(y)
        n_classes = class_labs.shape[0]
        m = len(y)
        y_encoded = np.zeros((m, n_classes))
        for i, val in enumerate(class_labs):
            y_encoded[y==val,i]=1
        return y_encoded
    
    def fit(self, X, y):
        y_encoded = self.one_hot(y)
        self.thetas = np.zeros([X.shape[1]+1,y_encoded.shape[1]])
        
        for i in range(y_encoded.shape[1]):
            temp = self.batch_gradient_fit(X, y_encoded[:,i]).reshape(1,3)
            self.thetas[i] =  temp
                
    def batch_gradient_fit(self, X, y):
        X = np.c_[np.ones((X.shape[0],1)),X]
        m, k = X.shape
        theta = np.random.randn(k,1)

        for iteration in range(self.n_iters):
            gradients = self.delJ(m, X, theta, y)
            theta = theta  - self.eta * gradients
        return np.array(theta)

    def predict_proba_v1(self, X):
        yhat_init = False
        
        for theta in self.thetas:
            if yhat_init:
                yhat = np.c_[ yhat,self.log_reg_predict_proba( X, theta)]
            else:
                yhat = self.log_reg_predict_proba( X, theta)
                yhat_init= True
        return yhat
            
    
    def predict_proba(self, X):
        X = np.c_[np.ones((X.shape[0],1)),X]
        p_hat = self.vsigma(self.thetas.dot(X.T)).T
        return p_hat
    
    def log_reg_predict_proba(self, X, theta):
        X = np.c_[np.ones((X.shape[0],1)),X]
        p_hat = self.vsigma(X.dot(theta))
        return p_hat
    
    

my_logreg = MyLogReg()
# my_logreg.batch_gradient_fit(X, y1)


my_logreg.fit(X,y)
my_logreg.thetas
print('')
print(my_logreg.predict_proba_v1(X))
print('')
print(my_logreg.predict_proba(X[:,:]))






[[  9.20518946e-01   2.36687434e-01   1.01822556e-02]
 [  9.20518946e-01   2.36687434e-01   1.01822556e-02]
 [  9.30508220e-01   2.23265384e-01   1.00604317e-02]
 [  9.09233806e-01   2.50655990e-01   1.03055393e-02]
 [  9.20518946e-01   2.36687434e-01   1.01822556e-02]
 [  8.41632716e-01   2.30255213e-01   1.88117567e-02]
 [  9.07002968e-01   2.13725313e-01   1.36023418e-02]
 [  9.09233806e-01   2.50655990e-01   1.03055393e-02]
 [  9.20518946e-01   2.36687434e-01   1.01822556e-02]
 [  9.22453280e-01   2.76192116e-01   7.70790265e-03]
 [  9.09233806e-01   2.50655990e-01   1.03055393e-02]
 [  8.96526457e-01   2.65162549e-01   1.04302999e-02]
 [  9.32217346e-01   2.61296712e-01   7.61545501e-03]
 [  9.55065337e-01   2.19827642e-01   7.34466302e-03]
 [  9.39324782e-01   2.10394675e-01   9.94005079e-03]
 [  8.76601533e-01   2.04485569e-01   1.83680916e-02]
 [  9.04723045e-01   1.80922126e-01   1.79346989e-02]
 [  9.07002968e-01   2.13725313e-01   1.36023418e-02]
 [  8.63216667e-01   2.5441

In [26]:
class_labs = np.unique(y)
n_classes = class_labs.shape[0]
m = len(y)
y_encoded = np.zeros((m, n_classes))
for i, val in enumerate(class_labs):
    y_encoded[y==val,i]=1

In [33]:
y_encoded.shape

(150, 3)